In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
target_size = 30*5
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")
model_max_length = 4096

In [4]:
with open("input/student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [5]:
df = pd.read_csv("input/geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [7]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 4min 8s, sys: 3min 38s, total: 7min 46s
Wall time: 13min 34s


In [8]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [9]:
prompter = Prompter(target_size=target_size, student_names=student_names, street_addresses=street_addresses)
res = prompter.prompt()
print(res.prompt)

[INST] Write a short essay on the following topic:
        In Education Policy and Reform, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?
        Intersperse the following information in different parts of the essay:
        1. Give an example that occurred near Tampines Street 82 (Block 839), at a school frequented by Fulvio Galiazzo
        2. Give an example citing someone named FulvioGaliazzo
        3. Give an example citing: periscope.tv/officialfulvioiamgaliazzo [/INST]
        


In [10]:
%%time
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [4:53:22<00:00, 117.35s/it]

CPU times: user 4h 36min 51s, sys: 16min 35s, total: 4h 53min 27s
Wall time: 4h 53min 22s


In [11]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            150 non-null    object
 1   prompt          150 non-null    object
 2   street_address  150 non-null    object
 3   student_name    150 non-null    object
 4   username        150 non-null    object
 5   personal_url    150 non-null    object
 6   model           150 non-null    object
dtypes: object(7)
memory usage: 8.3+ KB


In [12]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In the field of architecture and urban design, visualization tools have become increasingly important for fostering innovation and collaboration in a design thinking approach. These tools allow designers to communicate their ideas more effectively, test different concepts quickly, and collaborate with stakeholders throughout the design process.\n\n One example of how visualization tools have been used to foster innovation and collaboration can be seen in a project near Choa Chu Kang Avenue 4, at a nature park frequented by Rahman bin Hassan. In this project, architects and urban designers utilized virtual reality (VR) technology to create immersive, 360-degree visualizations of the proposed park redesign. This allowed both the design team and community members like Rahman to explore the space in a realistic way, providing valuable feedback and insights that ultimately improved the final design.\n\n Another example comes from the work of designer _4807.rahman-iam--hassan, who has used digital sketching and modeling tools to prototype new concepts for modular housing units. By using these visualization tools early in the design process, _4807.rahman-iam--hassan was able to quickly iterate on different designs, test them in various configurations, and share them with stakeholders in order to gather feedback and refine the concept.\n\n Periscope.tv/rahmanhassanxxx is another platform where Rahman Hassan has showcased his innovative design ideas, specifically focused on sustainable and affordable housing. He uses a combination of hand-drawn sketches and digital models to bring his ideas to life, allowing him to easily experiment with different materials, textures, and colors. The use of these visualization tools enables Rahman to communicate complex ideas clearly and efficiently, fostering greater collaboration among stakeholders and leading to more innovative solutions.\n\n In conclusion, visualization tools are crucial for driving innovation and collaboration in architecture and urban design projects. They enable designers to create accurate representations of their ideas, facilitating communication between team members and stakeholders. With the help of tools such as VR, digital sketching, and prototyping platforms, professionals like Rahman bin Hassan and _4807.rahman-iam--hassan continue to push the boundaries of what's possible in their fields, creating spaces and structures that not only meet functional needs but also inspire and engage those who interact with them.","[INST] Write a short essay on the following topic:\n In Architecture and Urban Design, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Choa Chu Kang Avenue 4 (Block 444), at a nature park frequented by Rahman bin Hassan\n 2. Give an example citing someone named _4807.rahman-iam--hassan\n 3. Give an example citing: periscope.tv/rahmanhassanxxx [/INST]\n",Choa Chu Kang Avenue 4 (Block 444),Rahman bin Hassan,_4807.rahman-iam--hassan,periscope.tv/rahmanhassanxxx,Mixtral-8x7B-Instruct-v0.1
1,"In sustainable urban development, storytelling is a powerful tool that can drive innovation and collaboration among stakeholders. By sharing stories about people's experiences and perspectives, designers and planners can gain insights into the needs and aspirations of the community, which can inform the design thinking process and lead to more inclusive and sustainable outcomes. Here are some examples of how storytelling has been used to enhance sustainable urban development:\n\nAt a subway station near Jurong West Street 92 (Block 922), a group of residents noticed that commuters often encountered difficulties when using the station's narrow staircase during peak hours. To address this issue, they worked with the transportation authority and local d

In [13]:
df.to_parquet("output/part01.parquet")
assert df.notna().all(axis=None)

In [14]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 5:06:58.222025
